In [ ]:
import tensorflow as tf
import numpy as np
import h5py as h5
import random as rd

In [ ]:
h5f = '/home/ivkhar/Baikal/data/mc_baikal_norm_cut-0_ordered_equal_big.h5'

max_len = 110
file = h5f 
regime = 'train' 
batch_size = 32
return_reminder = True 
k_nearest = 2

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# generator without shuffling
# yields (data, labels, adjacency)
class generator_no_shuffle:
    
    def __init__(self, file, regime, batch_size, return_reminder, k_nearest):
        self.file = file
        self.regime = regime
        self.batch_size = batch_size
        self.return_reminder = return_reminder
        with h5.File(self.file,'r') as hf:
            self.num = hf[self.regime+'/data'].shape[0]
            self.data_length = hf[self.regime+'/data'].shape[1]     
        self.batch_num = self.num // self.batch_size
        if return_reminder:
            self.gen_num = self.num
        else:
            self.gen_num = self.batch_num*self.batch_size
        te = [ np.expand_dims(np.eye(self.data_length),axis=0)]
        for i in range(1,k_nearest):
            te.append(np.expand_dims(np.eye(self.data_length, k=i),axis=0))
            te.append(np.expand_dims(np.eye(self.data_length, k=-i),axis=0))
        self.full_adj = np.sum( np.concatenate(te, axis=0), axis=0 )

    def __call__(self):
        start = 0
        stop = self.batch_size
        with h5.File(self.file, 'r') as hf:
            for i in range(self.batch_num):
                mask = hf[self.regime+'/mask'][start:stop]
                mask_channel = np.expand_dims( mask, axis=-1 )
                mask_channel_2 = np.expand_dims( mask, axis=1 )
                mask_to_adj = mask_channel*mask_channel_2
                labels = hf[self.regime+'/labels_signal_noise'][start:stop]
                yield ( np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel), axis=-1), 
                  labels, self.full_adj*mask_to_adj )
                start += self.batch_size
                stop += self.batch_size
            if self.return_reminder:
                mask = hf[self.regime+'/mask'][start:stop]
                mask_channel = np.expand_dims( mask, axis=-1 )
                mask_channel_2 = np.expand_dims( mask, axis=1 )
                mask_to_adj = mask_channel*mask_channel_2
                labels = hf[self.regime+'/labels_signal_noise'][start:stop]
                yield ( np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel), axis=-1), 
                  labels, self.full_adj*mask_to_adj )
                
# generator with shuffling
class generator_with_shuffle:
    
    def __init__(self, file, regime, batch_size, buffer_size, return_reminder, k_nearest):
        self.file = file
        self.regime = regime
        self.batch_size = batch_size
        self.return_reminder = return_reminder
        self.buffer_size = buffer_size
        with h5.File(self.file,'r') as hf:
            self.num = hf[self.regime+'/data/'].shape[0]
            self.data_length = hf[self.regime+'/data'].shape[1]
        self.batch_num = (self.num-self.buffer_size) // self.batch_size
        self.last_batches_num = self.buffer_size // self.batch_size
        if return_reminder:
            self.gen_num = self.num
        else:
            self.gen_num = (self.batch_num+self.last_batches_num)*self.batch_size
        te = [ np.expand_dims(np.eye(self.data_length),axis=0)]
        for i in range(1,k_nearest):
            te.append(np.expand_dims(np.eye(self.data_length, k=i),axis=0))
            te.append(np.expand_dims(np.eye(self.data_length, k=-i),axis=0))
        self.full_adj = np.sum( np.concatenate(te, axis=0), axis=0 )

    def __call__(self):
        start = self.buffer_size
        stop = self.buffer_size + self.batch_size
        with h5.File(self.file, 'r') as hf:
            mask = hf[self.regime+'/mask'][:self.buffer_size] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            mask_channel = np.expand_dims( mask, axis=-1 ) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            mask_channel_2 = np.expand_dims( mask, axis=1 ) ##!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            buffer_data = np.concatenate((hf[self.regime+'/data'][:self.buffer_size],mask_channel), axis=-1) #
            
            buffer_labels = hf[self.regime+'/labels_signal_noise'][:self.buffer_size] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            buffer_adj = self.full_adj*mask_channel*mask_channel_2 ####
            for i in range(self.batch_num): #
                idxs = rd.sample( range(self.buffer_size), k=self.batch_size ) #
                yield ( buffer_data[idxs], buffer_labels[idxs], buffer_adj[idxs] )  #
                mask = hf[self.regime+'/mask'][start:stop] #
                mask_channel = np.expand_dims( mask, axis=-1 ) #
                mask_channel_2 = np.expand_dims( mask, axis=1 ) #
                buffer_data[idxs] = np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel),axis=-1)#
                labels = hf[self.regime+'/labels_signal_noise'][start:stop] #
                buffer_labels[idxs] = labels #
                adj = self.full_adj*mask_channel*mask_channel_2#
                buffer_adj[idxs] = adj
                start += self.batch_size##
                stop += self.batch_size
            # fill the buffer with left data, if any
            mask = hf[self.regime+'/mask'][start:stop]###
            mask_channel = np.expand_dims( mask, axis=-1 ) #
            mask_channel_2 = np.expand_dims( mask, axis=1 ) #
            buffer_data = np.concatenate( (buffer_data,np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel),axis=-1)), axis=0 )
            labels = hf[self.regime+'/labels_signal_noise'][start:stop] ##########################
            buffer_labels = np.concatenate( (buffer_labels,labels), axis=0 )
            adj = self.full_adj*mask_channel*mask_channel_2
            buffer_adj = np.concatenate( (buffer_adj,adj), axis=0 )
            sh_idxs = rd.sample( range(buffer_labels.shape[0]), k=buffer_labels.shape[0] )
            start = 0
            stop = self.batch_size
            for i in range(self.last_batches_num):
                idxs = sh_idxs[start:stop]
                yield ( buffer_data[idxs], buffer_labels[idxs], buffer_adj[idxs] )
                start += self.batch_size
                stop += self.batch_size
            if self.return_reminder:
                idxs = sh_idxs[start:stop]
                yield ( buffer_data[idxs], buffer_labels[idxs], buffer_adj[idxs] )

In [ ]:
### Datasets
def make_datasets(h5f,make_generator_shuffle,return_batch_reminder,train_batch_size,train_buffer_size,test_batch_size,k_nearest):
    # generator for training data
    if make_generator_shuffle:
        tr_generator = generator_with_shuffle(h5f,'train',train_batch_size,train_buffer_size,return_batch_reminder,k_nearest)
    else:
        tr_generator = generator_no_shuffle(h5f,'train',train_batch_size,return_batch_reminder,k_nearest)
    if return_batch_reminder:
        # size of the last batch is unknown
        tr_batch_size = None
    else:
        tr_batch_size = train_batch_size

    train_dataset = tf.data.Dataset.from_generator( tr_generator, 
                        output_signature=( tf.TensorSpec(shape=(tr_batch_size,max_len,6)), tf.TensorSpec(shape=(tr_batch_size,max_len,2)),
                                         tf.TensorSpec(shape=(tr_batch_size,max_len,max_len))) )

    if make_generator_shuffle:
        train_dataset = train_dataset.repeat(-1).prefetch(tf.data.AUTOTUNE)
    else:
        train_dataset = train_dataset.repeat(-1).shuffle(num_batch_shuffle)

    # generator for validation data
    te_generator = generator_no_shuffle(h5f,'test',test_batch_size,False,k_nearest)
    te_batch_size = tr_batch_size
    test_dataset = tf.data.Dataset.from_generator( te_generator, 
                        output_signature=( tf.TensorSpec(shape=(tr_batch_size,max_len,6)), tf.TensorSpec(shape=(tr_batch_size,max_len,2)),
                                          tf.TensorSpec(shape=(tr_batch_size,max_len,max_len))) )
    
    test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

    return train_dataset, test_dataset

In [ ]:
train_dataset, test_dataset = make_datasets(h5f,True,False,batch_size,1000*batch_size,batch_size,k_nearest)

In [ ]:
class GraphConvLayer(tf.keras.layers.Layer):
    
    def __init__(self, units, activation):
        super(GraphConvLayer, self).__init__()
        self.lr = tf.keras.layers.Dense(units)
        self.act = activation

    def call(self, data, adj):
        # linea transform features
        transf_fts = self.lr(data)
        # aggregate information via adj matrix
        ret_fts = tf.matmul(adj, transf_fts)
        #print(ret_fts.shape)
        return self.act(ret_fts)

In [ ]:
# no attention
class GraphConvModel(tf.keras.Model):
    
    def __init__(self, hid_units_s, hid_act, out_units, out_act):
        super(GraphConvModel, self).__init__()
        self.hid_layers = []
        for un in hid_units_s:
            self.hid_layers.append(GraphConvLayer(un, hid_act))
        self.out_layer = GraphConvLayer(out_units, out_act)
        
    def compile(self, optimizer, loss_fn, metrics):
        super(GraphConvModel, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.loss_tracker = tf.keras.metrics.Mean(name='loss')
        self.metrics_ = metrics
        self.all_metrics = metrics+[self.loss_tracker]
        
    def norm_adj(self, adj):
        degree = tf.reduce_sum(adj, axis=-1)
        norm_degree = tf.linalg.diag(1./tf.sqrt(degree))
        n_adj = tf.matmul(norm_degree, tf.matmul(adj, norm_degree) )
        return n_adj
    
    @tf.function
    def call(self, datas):
        (x, labels, adj) = datas
        adj = self.norm_adj(adj)
        mask = x[:,:,-1:]
        for gr_lr in self.hid_layers:
            x = gr_lr(x, adj)
        x = self.out_layer(x, adj)
        preds = tf.where( tf.cast(mask,bool), tf.keras.layers.Softmax(axis=-1)(x), tf.constant([0.,1.]) )
        return preds
        
    @property
    def metrics(self):
        return self.all_metrics
    
    @tf.function
    def train_step(self, datas):
        (x, labels, adj) = datas
        with tf.GradientTape() as tape:
            preds = self.call(datas)
            loss = self.loss_fn(labels,preds)
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients( zip(grads, self.trainable_weights) )
        self.loss_tracker.update_state(loss)
        prd_cls = tf.math.argmax( preds, axis=-1 )
        true_cls = tf.math.argmax( labels, axis=-1 )
        for m in self.metrics_:
            m.update_state(prd_cls, true_cls)
        ms = { m.name : m.result() for m in self.all_metrics }
        return ms
    
    @tf.function
    def test_step(self, datas):
        (x, labels, adj) = datas
        preds = self.call(datas)
        loss = self.loss_fn(labels,preds)
        self.loss_tracker.update_state(loss)
        prd_cls = tf.math.argmax( preds, axis=-1 )
        true_cls = tf.math.argmax( labels, axis=-1 )
        for m in self.metrics_:
            m.update_state(prd_cls, true_cls)
        ms = { m.name : m.result() for m in self.all_metrics }
        return ms

In [ ]:
hid_units_s = [8]
hid_act = tf.keras.activations.selu
out_units = 2 
out_act = tf.keras.activations.softmax

gnn = GraphConvModel(hid_units_s, hid_act, out_units, out_act)

lr = 0.001
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
metrics = [tf.keras.metrics.Accuracy()]
gnn.compile(optimizer, loss_fn, metrics)

In [ ]:
## why test loss fixed???
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=100, validation_data=test_dataset, verbose=1)

In [3]:
75+180-4

251

In [6]:
x**5-x**4+x**3-x**2+1

0.7401599999999999

In [4]:
x=-0.4

In [20]:
s = input().split(".")
if len(s) != 4:
    answer = "false"
else:
    answer = "true"
    for i in s:
        if float(i)%1 != 0 or int(i)<0 or int(i)>255:
            answer = "false"
print(answer)


-10.-20.-30.-40
false


In [ ]:
d={'I':'1','V':'5','X':'10','L':'50','C':'100','M':'1000'}

In [27]:
word = input()
d = {"I":1,"V":5,"X":10,"L":50,"C":100,"D":500,"M":1000}
arab = 0
for i in range(len(word)):
    try:
        if d[word[i]] < d[word[i+1]]:
            arab -= d[word[i]]
        else:
            arab += d[word[i]]
    except KeyError:
        arab = 0
        break
    except IndexError:
        arab += d[word[i]]
print(arab)

HELLO
0


In [ ]:
def f(a):return list(a[0])+f(zip(*a[1:])[::-1])if a else []

In [93]:
n = int(input())
array =[]
for i in range(n):
    array.append(list(map(int,input().split(" "))))
#print(array)

direction=[-1,0]
for i in range(0,n-1):
    a= [array[j][i] for j in range(i,n-1)]
    print(a*,end = " ")
    a = [array[-(i+1)][j] for j in range(i,n-1-i)]
    print(a*,end = " ")
    a = [array[j][-(i+1)] for j in range(n-1-i,i,-1)]
    print(a*,end = " ")
    a = [array[i][j] for j in range(n-i-1,i,-1)]
    print(a*,end = " ")
    

3
63 83 3
23 43 22
33 44 55
63 23 33 44 55 22 3 83 43 

In [91]:
i=0
print( [array[j][i] for j in range(i,n-1)], end = " ")
print([array[-(i+1)][j] for j in range(i,n-1-i)], end = " ")
print( [array[j][-(i+1)] for j in range(n-1-i,i,-1)], end = " ")
print(  [array[i][j] for j in range(n-i-1,i,-1)], end = " ")

[63, 23] [33, 44] [55, 22] [3, 83] 

In [66]:
array

[[63, 83, 3], [23, 43, 22], [33, 44, 55]]

In [82]:
[array[0][i] for i in range(2)]

[63, 83]

In [42]:
from math import gcd
s = input()
first,sign,second = s.split(" ")
first_up ,first_down = list(map(int,first.split("/")))
second_up ,second_down = list(map(int,second.split("/")))
#print(first_up ,first_down,second_up ,second_down,sign)
if sign == "+":
    numerator =  first_up * second_down + second_up * first_down
    denominator = first_down * second_down
elif sign == "-":
    numerator =  first_up * second_down - second_up * first_down
    denominator = first_down * second_down    
elif sign == "*":
    numerator =  first_up * second_up
    denominator = first_down * second_down
elif sign == "/":
    numerator =  first_up * second_down
    denominator = first_down * second_up 
nod = gcd(numerator, denominator)
numerator //= nod
denominator //= nod

print(s+" =",str(numerator)+"/"+str(denominator))

2/3 / 3/7
2/3 / 3/7 = 14/9


In [46]:
s = input()
n = 0
answer = 'true'
for i in s:
    if i == "(":
        n += 1
    else :
        n -= 1
    if n < 0:
        answer = "false"
        break
if n != 0:
    print('false')
else:
    print(answer)

())(()
false
